In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [21]:
data_dir = "tiny-imagenet-200/train"
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),      
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

print(f"Dataset size: {len(train_dataset)}")

Dataset size: 100000


In [3]:
x = torch.randn(64, 3, 64, 64)
x.shape

torch.Size([64, 3, 64, 64])

In [14]:
class Small_AlexNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes

        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11),
            nn.ReLU(),
            # BatchNorm here, try later
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, 5, padding=2),
            nn.ReLU(),
            # BatchNorm here, try later
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            # add Dropout(0.2)
            nn.Linear(in_features=(128*5*5), out_features=1024),
            nn.ReLU(),
            # add Dropout(0.2)
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, out_features=self.num_classes)
        )

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 128*5*5)
        x = self.classifier(x)
        return x

In [15]:
model = Small_AlexNet(num_classes=200)

x = torch.randn(64, 3, 64, 64)
x = model(x)
x.shape

torch.Size([64, 200])

In [16]:
sum(p.numel() for p in model.parameters())

5940936

In [22]:
count = 0
for idx, (image, label) in enumerate(train_loader):
    count += 1

print(count)

100
